### Import General Python libs, defined classes, and modules¶

In [1]:
from pprint import pprint
import mlflow
from mlflow.tracking import MlflowClient
from train_model import DriverRankingTrainModel
from predict_model import DriverRankingPredictModel

### Train the model

 * Use the `DriverRankingTrainingModel` class, which fetches training data from the Feast data source
 * Use `mlflow.autolog` to automatically log the parameters and computed metrics during training
 * Use `mflow` Fluent APIs to log the Feast training set

### Define ElasticNet tuning parameters and run some experiments

In [2]:
params_list = [{"alpha": 0.5, "l1_ratio": 0.15},
               {"alpha": 0.75, "l1_ratio": 0.25},
               {"alpha": 1.0, "l1_ratio": 0.5}]

# Change this to your location for the Feast feature repo
REPO_PATH = "/Users/jules/git-repos/feast_workshops/module_1/feature_repo"
# define your feature service name
FEATURE_SERVICE_NAME = "driver_ranking_fv_svc"

### Iterate and experiment MLflow runs tunning parameters

In [3]:
# iterate over tuning parameters
for params in params_list:
    model_cls = DriverRankingTrainModel(REPO_PATH, FEATURE_SERVICE_NAME, params)
    run_id = model_cls.train_model()
    pprint(f"ElasticNet params: {params}")
    print(f"Model run id: {run_id}")

2021/08/27 08:51:48 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2021/08/27 08:51:48 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
/Users/jules/miniconda3/lib/python3.9/contextlib.py:124: SADeprecationWarning: The Column.copy() method is deprecated and will be removed in a future release. (deprecated since: 1.4)
  next(self.gen)
/Users/jules/miniconda3/lib/python3.9/contextlib.py:124: SADeprecationWarning: The ColumnCollectionConstraint.copy() method is deprecated and will be removed in a future release. (deprecated since: 1.4)
  next(self.gen)
/Users/jules/miniconda3/lib/python3.9/contextlib.py:124: SADeprecationWarning: The ForeignKeyConstraint.copy() method is deprecated and will be removed in a future release. (deprecated since: 1.4)
  next(

"ElasticNet params: {'alpha': 0.5, 'l1_ratio': 0.15}"
Model run id: {'62fb2746b80c40f485cfb9dcbcfb366a'}


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2021/08/27 08:51:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/jules/miniconda3/lib/python3.9/site-packages/mlflow/types/utils.py:139: DeprecationWarning: .. Warning:: ``mlflow.types.schema.column_types`` is deprecated since 1.14. This method will be removed in a near future release. Use ``mlflow.types.Schema.input_types`` instead."
2021/08/27 08:51:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/jules/miniconda3/lib/python3.9/site-packages/mlflow/types/schema.py:366: DeprecationWarning: .. Warning:: ``mlflow.types.schema.columns`` is deprecated since 1.14. This method will be removed in a near future release. Use ``mlflow.types.Schema.inputs``

"ElasticNet params: {'alpha': 0.75, 'l1_ratio': 0.25}"
Model run id: {'75052b557f0a4c8c9e24408c573a3056'}


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2021/08/27 08:51:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/jules/miniconda3/lib/python3.9/site-packages/mlflow/types/utils.py:139: DeprecationWarning: .. Warning:: ``mlflow.types.schema.column_types`` is deprecated since 1.14. This method will be removed in a near future release. Use ``mlflow.types.Schema.input_types`` instead."
2021/08/27 08:51:50 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/jules/miniconda3/lib/python3.9/site-packages/mlflow/types/schema.py:366: DeprecationWarning: .. Warning:: ``mlflow.types.schema.columns`` is deprecated since 1.14. This method will be removed in a near future release. Use ``mlflow.types.Schema.inputs``

"ElasticNet params: {'alpha': 1.0, 'l1_ratio': 0.5}"
Model run id: {'e7270456bc7145d4a58719ea92f17b76'}


### Launch the MLflow ui, with Model Registry at the local SQLite database
 * Navigate and examine runs for the model
 * Register the best model with lowest RMSE with the Model Registry

In [6]:
!mlflow ui  --backend-store-uri sqlite:///mlruns.db

[2021-08-27 08:55:14 -0700] [83866] [INFO] Starting gunicorn 20.1.0
[2021-08-27 08:55:14 -0700] [83866] [INFO] Listening at: http://127.0.0.1:5000 (83866)
[2021-08-27 08:55:14 -0700] [83866] [INFO] Using worker: sync
[2021-08-27 08:55:14 -0700] [83867] [INFO] Booting worker with pid: 83867
^C
[2021-08-27 08:57:51 -0700] [83866] [INFO] Handling signal: int
[2021-08-27 08:57:51 -0700] [83867] [INFO] Worker exiting (pid: 83867)


### CI/CD Intergration: Fetch the registered model from the Model Registry
 * Use the model URI (either by stage or version)
 * Make the predicion

In [7]:
mlflow.set_tracking_uri("sqlite:///mlruns.db")
REPO_PATH = "/Users/jules/git-repos/feast_workshops/module_1/feature_repo"
model_uri = "models:/sklearn_feast_integration/staging"
model = DriverRankingPredictModel(REPO_PATH, model_uri, FEATURE_SERVICE_NAME)
drivers = [1001, 1002, 1003]
best_driver = model(drivers)
print(f" Best predicted driver for completed trips: {best_driver}")

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


 Best predicted driver for completed trips: 1001


### CI/CD Intergration: Transition the model to production 

In [8]:
client = MlflowClient()
client.transition_model_version_stage(
    name="sklearn_feast_integration",
    version=1,
    stage="Production"
)

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<ModelVersion: creation_timestamp=1630079837363, current_stage='Production', description='', last_updated_timestamp=1630079901057, name='sklearn_feast_integration', run_id='62fb2746b80c40f485cfb9dcbcfb366a', run_link='', source='./mlruns/0/62fb2746b80c40f485cfb9dcbcfb366a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>